<a href="https://colab.research.google.com/github/EnriqueProjectsIA/Pytorch-teaching/blob/main/cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
from matplotlib import pyplot as plt

import time

* It is possible to check if CUDA is available

In [2]:
torch.cuda.is_available()

True

* One can also check the available device ID

In [ ]:
torch.cuda.current_device()

0

* One can also find out the hardware name or the amount of RAM memory allocated:

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
torch.cuda.memory_allocated() # units in bits per device

0

* One can also check the memory management

In [ ]:
torch.cuda.memory_cached()

/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:393: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


0

In [ ]:
torch.cuda.memory_reserved()

0

## We create a tensor

In [ ]:
a = torch.FloatTensor([1.0,2.0,3.0,4.0,5.0])
a

tensor([1., 2., 3., 4., 5.])

In [ ]:
a.device

device(type='cpu')

* The tensor has been stored in the CPU and not in the GPU. This is because we have to specify it.

In [ ]:
a = torch.FloatTensor([1.0,2.0,3.0,4.0,5.0]).cuda()

In [ ]:
a.device

device(type='cuda', index=0)

* Now we can check for the amount of memory allocated.

In [ ]:
torch.cuda.memory_allocated()

512

* Therefore, if one wants to use CUDA, it is necessary to create the tensors (our data) using the method cuda.

## CUDA and models

* One generates the model

In [3]:
class Model(nn.Module):
  def __init__(self, in_features = 4, h1 = 8, h2 = 9, out_features = 3):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)
    return x

In [4]:
torch.manual_seed(42)
model = Model()

* One can check if the model parameters are on CUDA.

In [ ]:
next(model.parameters()).is_cuda

False

In [5]:
gpumodel = model.cuda()

In [6]:
next(gpumodel.parameters()).is_cuda

True

## Now we can use a toy Dataset

In [7]:
cols = ['sepal length','sepal width','petal length','petal width','class']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',names=cols, header=None)

In [8]:
df.tail()

,sepal length,sepal width,petal length,petal width,class
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [9]:
df['class'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [10]:
df['target'] = df.apply(lambda x: 0 if x['class'] == 'Iris-setosa' else 1 if \
                        x['class'] == 'Iris-versicolor' else 2, axis = 1)

In [11]:
df['target'].unique()

array([0, 1, 2])

In [21]:
X = df[['sepal length','sepal width','petal length','petal width']].values

In [ ]:
df['Iris-setosa'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [22]:
Y = df['target'].values

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [24]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.FloatTensor(y_train).type(torch.LongTensor).cuda()
y_test = torch.FloatTensor(y_test).type(torch.LongTensor).cuda()

In [17]:
trainloader = DataLoader(X_train, batch_size = 60, shuffle=True, pin_memory = True)
testloader = DataLoader(X_test, batch_size = 60, shuffle=False, pin_memory = True)

In [25]:
torch.manual_seed(42)
model = Model()
gpumodel = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

start = time.time()
epochs = 100
losses = []
for i in range(epochs):

  y_pred = gpumodel.forward(X_train)
  loss = criterion(y_pred,y_train)
  losses.append(loss)

  if i%10 ==0:
    print(f'Epoch: {i+1}, loss: {loss.item()}')

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()


end = time.time()-start
print(f'Total time: {end}')

Epoch: 1, loss: 1.0846846103668213
Epoch: 11, loss: 0.9183797836303711
Epoch: 21, loss: 0.7622429132461548
Epoch: 31, loss: 0.627383291721344
Epoch: 41, loss: 0.46674731373786926
Epoch: 51, loss: 0.2877061069011688
Epoch: 61, loss: 0.16738925874233246
Epoch: 71, loss: 0.10598890483379364
Epoch: 81, loss: 0.0802135318517685
Epoch: 91, loss: 0.06947017461061478
Total time: 0.20412540435791016


In [27]:
correct = 0
with torch.no_grad():
  for i,data in enumerate(X_test):
    y_val = gpumodel.forward(data)
    print(f'{i+1:2}. {str(y_val):38} {y_test[i]}')
    if y_val.argmax().item() == y_test[i]:
      correct += 1
print(f'\n{correct} out of {len(y_test)} = {100*correct/len(y_test):.2f}% correct')

 1. tensor([-1.4120,  6.3132,  2.6088], device='cuda:0') 1
 2. tensor([ 8.4023,  2.4573, -7.5391], device='cuda:0') 0
 3. tensor([-12.1991,   1.1829,  13.7443], device='cuda:0') 2
 4. tensor([-1.6752,  5.9271,  2.8622], device='cuda:0') 1
 5. tensor([-1.5887,  6.7559,  2.8371], device='cuda:0') 1
 6. tensor([ 7.8451,  2.3289, -7.0010], device='cuda:0') 0
 7. tensor([ 1.1273,  6.0707, -0.0493], device='cuda:0') 1
 8. tensor([-5.2088,  4.4484,  6.5316], device='cuda:0') 2
 9. tensor([-3.2614,  4.9128,  4.4788], device='cuda:0') 1
10. tensor([0.5683, 6.5565, 0.5445], device='cuda:0') 1
11. tensor([-4.3492,  4.7780,  5.6344], device='cuda:0') 2
12. tensor([ 7.3631,  2.0552, -6.5497], device='cuda:0') 0
13. tensor([ 8.1384,  2.3552, -7.2879], device='cuda:0') 0
14. tensor([ 7.4936,  2.1057, -6.6740], device='cuda:0') 0
15. tensor([ 7.8643,  2.2491, -7.0269], device='cuda:0') 0
16. tensor([-1.3322,  6.5325,  2.5448], device='cuda:0') 1
17. tensor([-8.2712,  2.4244,  9.6405], device='cuda:0')